In [ ]:
### Initialize Spark Session
import pyspark
from pyspark.sql import SparkSession
from delta.tables import *
S3_ENDPOINT = "192.168.132.131:9000"
S3_BUCKET = "sales"
S3_ACCESS_KEY="minioadmin"
S3_SECRET_KEY="minioadmin"

# This cell may take some time to run the first time, as it must download the necessary spark jars
conf = pyspark.SparkConf().setMaster("local")
conf.set("spark.jars.packages", 'org.apache.hadoop:hadoop-aws:3.3.1,io.delta:delta-spark_2.12:3.0.0')\
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")\
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')\
    .set('spark.hadoop.fs.s3a.endpoint', S3_ENDPOINT)\
    .set('spark.hadoop.fs.s3a.access.key', S3_ACCESS_KEY)\
    .set('spark.hadoop.fs.s3a.secret.key', S3_SECRET_KEY)\
    .set('spark.hadoop.fs.s3a.path.style.access', "true")

sc = pyspark.SparkContext(conf=conf)


spark = pyspark.sql.SparkSession(sc)



# Write File or folder directly into MinIO bucket 

In [ ]:
# import os
# path="D:/test/"
# def get_file_list(path):
#     file_list = []
#     for root, dirs, files in os.walk(path):
#         for file in files:
#             file_list.append(os.path.join(root, file))
#     return file_list

# def fileOrFolder(path):
#     # check if file have . in file name return True if have . in file name
#     if "." in path:
#         return True
#     else:
#         return False

# def MinioRename(file_name): 
#     file_name=file_name.replace("/mnt/hgfs/","")
#     file_name= "Bronze/" + file_name
#     file_name=file_name.replace("\\","/")
#     file_name=file_name.replace(" ","_")
#     return file_name


In [ ]:
# # up load file to minio

# from minio import Minio
# from minio_client_functions import *
# from support_functions import *
# client = Minio(endpoint=S3_ENDPOINT,
#                 access_key=S3_ACCESS_KEY,
#                 secret_key=S3_SECRET_KEY,
#                 secure=False)

# client=client_initialize(S3_ENDPOINT,S3_ACCESS_KEY,S3_SECRET_KEY)
# upload_object(client,S3_BUCKET,path)

# Ingest data from SQL server to Bronze layer

In [287]:
server_name = "jdbc:sqlserver://192.168.1.71:1433"
database_name = "new_sales"
url = server_name + ";" + "databaseName=" + database_name + ";"
table_name = "dbo.sales"
jdbcDF = spark.read.format("com.microsoft.sqlserver.jdbc.spark")\
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
.option("url", url)\
.option("dbtable",table_name )\
.option("user", "sa")\
.option("password", "sa")\
.load()

In [288]:
try:
      deltaTable=DeltaTable.forPath(spark,f"s3a://{S3_BUCKET}/bronze")
      deltaTable.alias("sales").merge(
      jdbcDF.alias("updates"),
                      "sales.ID = updates.ID")                                \
                      .whenMatchedUpdate(set = {"ORDERID": 	    "updates.ORDERID", 	      \
                                                "ORDERDETAILID":"updates.ORDERDETAILID",      \
                                                "DATE_": "updates.DATE_",  \
                                                "USERID":       "updates.USERID",       \
                                                "USERNAME_": "updates.USERNAME_",\
                                                "NAMESURNAME": "updates.NAMESURNAME",
                                                "ITEMID":"updates.ITEMID",
                                                "ITEMCODE": "updates.ITEMCODE",
                                                "AMOUNT":"updates.AMOUNT",
                                                "PRICE":"updates.PRICE",
                                                "TOTALPRICE":"updates.TOTALPRICE",
                                                "CATEGORY1":"updates.CATEGORY1",
                                                "CATEGORY2":"updates.CATEGORY2",
                                                "CATEGORY3":"updates.CATEGORY3",
                                                "CATEGORY4":"updates.CATEGORY4",
                                                "BRAND":"updates.brand",
                                                "USERGENDER":"updates.USERGENDER",
                                                "USERBIRTHDATE":"updates.USERBIRTHDATE",
                                                "REGION":"updates.REGION",
                                                "CITY":"updates.CITY",
                                                "TOWN":"updates.TOWN",
                                                "DISTRICT":"updates.DISTRICT",
                                                "ADDRESSTEXT":"updates.ADDRESSTEXT"}
                      )\
                      .whenNotMatchedInsert(values =                                                  \
                         {                      "ID": "updates.ID",
                                                "ORDERID": 	    "updates.ORDERID", 	      \
                                                "ORDERDETAILID":     "updates.ORDERDETAILID",      \
                                                "DATE_": "updates.DATE_",  \
                                                "USERID":       "updates.USERID",       \
                                                "USERNAME_": "updates.USERNAME_",\
                                                "NAMESURNAME": "updates.NAMESURNAME",
                                                "ITEMID":"updates.ITEMID",
                                                "ITEMCODE": "updates.ITEMCODE",
                                                "AMOUNT":"updates.AMOUNT",
                                                "PRICE":"updates.PRICE",
                                                "TOTALPRICE":"updates.TOTALPRICE",
                                                "CATEGORY1":"updates.CATEGORY1",
                                                "CATEGORY2":"updates.CATEGORY2",
                                                "CATEGORY3":"updates.CATEGORY3",
                                                "CATEGORY4":"updates.CATEGORY4",
                                                "BRAND":"updates.BRAND",
                                                "USERGENDER":"updates.USERGENDER",
                                                "USERBIRTHDATE":"updates.USERBIRTHDATE",
                                                "REGION":"updates.REGION",
                                                "CITY":"updates.CITY",
                                                "TOWN":"updates.TOWN",
                                                "DISTRICT":"updates.DISTRICT",
                                                "ADDRESSTEXT":"updates.ADDRESSTEXT"}                                                                           \
                       ).execute() 
except:
      jdbcDF.write.format("delta").mode("overwrite").save(f"s3a://{S3_BUCKET}/bronze")
      print("Created Bronze layer successfully")

Created Bronze layer successfully


In [289]:
deltaTable=DeltaTable.forPath(spark,f"s3a://{S3_BUCKET}/bronze")
deltaTable.history().show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      0|2023-12-06 17:17:46|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|       NULL|  Serializable|        false|{numFiles -> 1, n...|        NULL|Apache-Spark/3.5....|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+



# Bronze To Silver

In [290]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df_bronze=(spark.read.format("delta")
            .option("header", "true")
            .load(f"s3a://{S3_BUCKET}/bronze"))

df_bronze.show()

+--------+--------+-------------+-------------------+------+--------------------+--------------------+------+--------+--------------------+------+-----+------------------+---------+--------------------+---------+-------------------+------------+----------+-------------+-----------------+-------------+--------------------+--------------------+--------------------+
|      ID| ORDERID|ORDERDETAILID|              DATE_|USERID|           USERNAME_|         NAMESURNAME|ITEMID|ITEMCODE|            ITEMNAME|AMOUNT|PRICE|        TOTALPRICE|CATEGORY1|           CATEGORY2|CATEGORY3|          CATEGORY4|       BRAND|USERGENDER|USERBIRTHDATE|           REGION|         CITY|                TOWN|            DISTRICT|         ADDRESSTEXT|
+--------+--------+-------------+-------------------+------+--------------------+--------------------+------+--------+--------------------+------+-----+------------------+---------+--------------------+---------+-------------------+------------+----------+------------

## Validate data 

### Check null values

In [291]:
df_bronze.select([count(when(col(c).isNull(), c)).alias(c) for c in df_bronze.columns]).show() 

+---+-------+-------------+-----+------+---------+-----------+------+--------+--------+------+-----+----------+---------+---------+---------+---------+-----+----------+-------------+------+----+----+--------+-----------+
| ID|ORDERID|ORDERDETAILID|DATE_|USERID|USERNAME_|NAMESURNAME|ITEMID|ITEMCODE|ITEMNAME|AMOUNT|PRICE|TOTALPRICE|CATEGORY1|CATEGORY2|CATEGORY3|CATEGORY4|BRAND|USERGENDER|USERBIRTHDATE|REGION|CITY|TOWN|DISTRICT|ADDRESSTEXT|
+---+-------+-------------+-----+------+---------+-----------+------+--------+--------+------+-----+----------+---------+---------+---------+---------+-----+----------+-------------+------+----+----+--------+-----------+
|  0|      0|            0|    0|     0|        0|          0|     0|       0|       0|     0|    0|         0|        0|        0|        0|        0|    0|         0|            0|     0|   0|   0|       0|          0|
+---+-------+-------------+-----+------+---------+-----------+------+--------+--------+------+-----+----------+-----

### Rename column name 

In [292]:
# Get the list of columns
cols = df_bronze.columns
df_validated=df_bronze
# Loop through each column and rename it
for col in cols:
  new_col = col.replace("_", "").capitalize()
  df_validated = df_validated.withColumnRenamed(col, new_col)

In [ ]:
df_validated.printSchema()

### Convert data type of specific attributes

In [293]:
df_validated=df_validated.withColumn("Date",to_date("Date"))
df_validated=df_validated.withColumn("Amount",(df_validated["Amount"]).cast('integer'))
df_validated=df_validated.withColumn("Totalprice",df_validated["Totalprice"].cast('decimal(10,2)'))
df_validated=df_validated.withColumn("Price",df_validated["Price"].cast('decimal(10,2)'))

In [294]:
df_validated.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Orderid: integer (nullable = true)
 |-- Orderdetailid: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Userid: integer (nullable = true)
 |-- Username: string (nullable = true)
 |-- Namesurname: string (nullable = true)
 |-- Itemid: integer (nullable = true)
 |-- Itemcode: string (nullable = true)
 |-- Itemname: string (nullable = true)
 |-- Amount: integer (nullable = true)
 |-- Price: decimal(10,2) (nullable = true)
 |-- Totalprice: decimal(10,2) (nullable = true)
 |-- Category1: string (nullable = true)
 |-- Category2: string (nullable = true)
 |-- Category3: string (nullable = true)
 |-- Category4: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Usergender: string (nullable = true)
 |-- Userbirthdate: date (nullable = true)
 |-- Region: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Town: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Addresstext: string (nulla

### Create schema by Object

In [295]:
df1 = df_validated.groupBy("Itemid").agg(max("Date").alias("Date"))

Item_silver = df_validated.join(df1, ["Itemid", "Date"]).select('Itemid','Itemcode','Itemname','Price','Category1','Category2','Category3','Category4','Brand')
Item_silver=Item_silver.drop_duplicates(subset=["Itemid"])

In [297]:

User_silver=df_validated.select('Userid','Username','Namesurname','Usergender','Userbirthdate','Region','City','Town','District','Addresstext').drop_duplicates(subset=["Userid"])
Order_silver=df_validated.select('Orderid','Userid','Date').drop_duplicates(subset=["Orderid"])
Orderdetail_silver=df_validated.select('Orderdetailid','Orderid','Itemid','Amount','Price','Totalprice').drop_duplicates(subset=["Orderdetailid"])

In [298]:
Item_silver_path=f"s3a://{S3_BUCKET}/silver/Item"
User_silver_path=f"s3a://{S3_BUCKET}/silver/User"
Order_silver_path=f"s3a://{S3_BUCKET}/silver/Order"
Orderdetail_silver_path=f"s3a://{S3_BUCKET}/silver/Orderdetail"

#Load Silver Item table
try:
      deltaTable=DeltaTable.forPath(spark,Item_silver_path)
      deltaTable.alias("item").merge(
      Item_silver.alias("updates"),
                      "item.Itemid = updates.Itemid")                                \
                      .whenMatchedUpdate(set = {
                                                "Itemcode": "updates.Itemcode",
                                                "Itemname":"updates.Itemname",
                                                "Amount":"updates.Amount",
                                                "Price":"updates.Price",
                                                "Totalprice":"updates.Totalprice",
                                                "Category1":"updates.Category1",
                                                "Category2":"updates.Category2",
                                                "Category3":"updates.Category3",
                                                "Category4":"updates.Category4",
                                                "Brand":"updates.Brand"
                                                }
                      )\
                      .whenNotMatchedInsert(values =                                                  \
                         {                      "Itemid": "updates.Itemid",
                                                "Itemcode": "updates.Itemcode",
                                                "Itemname":"updates.Itemname",
                                                "Amount":"updates.Amount",
                                                "Price":"updates.Price",
                                                "Totalprice":"updates.Totalprice",
                                                "Category1":"updates.Category1",
                                                "Category2":"updates.Category2",
                                                "Category3":"updates.Category3",
                                                "Category4":"updates.Category4",
                                                "Brand":"updates.brand"}                                                                           \
                       ).execute() 
      print("Inserted and updated Item successfully")
except:
      Item_silver.write.format("delta").mode("overwrite").save(Item_silver_path)
      print("Write Silver Item table successfully")


#Load Silver User table
try:
      deltaTable=DeltaTable.forPath(spark,User_silver_path)
      deltaTable.alias("user").merge(
      User_silver.alias("updates"),
                      "user.Userid = updates.Userid")                                \
                      .whenMatchedUpdate(set = {
                                                'Username': "updates.Username",
                                                'Namesurname': "updates.Namesurname",
                                                'Usergender': "updates.Usergender",
                                                'Userbirthdate':"updates.Userbirthdate",
                                                'Region':"updates.Region",
                                                'City':"updates.City",
                                                'Town':"updates.Town",
                                                'District':"updates.District",
                                                'Addresstext':"updates.Addresstext"
                                                }
                      )\
                      .whenNotMatchedInsert(values =                                                  \
                         {                      "Userid": "updates.Userid",
                                                'Username': "updates.Username",
                                                'Namesurname': "updates.Namesurname",
                                                'Usergender': "updates.Usergender",
                                                'Userbirthdate':"updates.Userbirthdate",
                                                'Region':"updates.Region",
                                                'City':"updates.City",
                                                'Town':"updates.Town",
                                                'District':"updates.District",
                                                'Addresstext':"updates.Addresstext"}                                                                           \
                       ).execute() 
      print("Inserted and updated User successfully")
except:
      User_silver.write.format("delta").mode("overwrite").save(User_silver_path)
      print("Write Silver User table successfully")



#Load Silver order table
try:
      deltaTable=DeltaTable.forPath(spark,Order_silver_path)
      deltaTable.alias("order").merge(
      Order_silver.alias("updates"),
                      "order.Orderid = updates.Orderid")                                \
                      .whenMatchedUpdate(set = {
                                                "Userid":"updates.Userid",
                                                "Date":"updates.Date"
                                                }
                      )\
                      .whenNotMatchedInsert(values =                                                  \
                         {                      "Orderid":"updates.Orderid",
                                                "Userid":"updates.Userid",
                                                "Date":"updates.Date"}                                                                           \
                       ).execute() 
      print("Inserted and updated Order successfully")
except:
      Order_silver.write.format("delta").mode("overwrite").save(Order_silver_path)
      print("Write Silver Order table successfully")


#Load Silver Orderdetail table
try:
      deltaTable=DeltaTable.forPath(spark,Orderdetail_silver_path)
      deltaTable.alias("orderdetail").merge(
      Orderdetail_silver.alias("updates"),
                      "orderdetail.Orderdetailid = updates.Orderdetailid")                                \
                      .whenMatchedUpdate(set = {
                                                'Orderid':"updates.Orderid",
                                                'Itemid':"updates.Itemid",
                                                'Amount':"updates.Amount",
                                                'Price':"updates.Price",
                                                'Totalprice':"updates.Totalprice"
                                                }
                      )\
                      .whenNotMatchedInsert(values =                                                  \
                         {                      'Orderdetailid':"updates.Orderdetailid",
                                                'Orderid':"updates.Orderid",
                                                'Itemid':"updates.Itemid",
                                                'Amount':"updates.Amount",
                                                'Price':"updates.Price",
                                                'Totalprice':"updates.Totalprice"}                                                                           \
                       ).execute()  
      print("Inserted and updated Orderdetail successfully")
except:
      Orderdetail_silver.write.format("delta").mode("overwrite").save(Orderdetail_silver_path)
      print("Write Silver Orderdetail table successfully")

Write Silver Item table successfully


Write Silver User table successfully


Write Silver Order table successfully


Write Silver Orderdetail table successfully


# Silver layer To Gold layer

In [299]:
df_order=spark.read.format("delta").option("header","true").load(f"s3a://{S3_BUCKET}/silver/Order")

In [300]:
df_orderdetail=spark.read.format("delta").option("header","true").load(f"s3a://{S3_BUCKET}/silver/Orderdetail")

In [301]:
totalpay_by_order=df_order.join(df_orderdetail.groupBy("Orderid").agg(sum('Totalprice').alias("TotalPay")),on=['Orderid'])

In [302]:
df_user=spark.read.format("delta").option("header","true").load(f"s3a://{S3_BUCKET}/silver/User")

In [303]:
totalpay_by_user=df_user.join(totalpay_by_order.groupBy("Userid").agg(sum("TotalPay").alias("TotalPay")).distinct(),on=["Userid"])

In [304]:
df_item=spark.read.format("delta").option("header","true").load(f"s3a://{S3_BUCKET}/silver/Item")

In [305]:
totalsold_totalamount_by_item=df_item.join(df_orderdetail.groupBy("Itemid").agg(sum('Amount').alias("SoldQuantity")),on=['Itemid']).distinct()

In [306]:
totalsold_totalamount_by_item=totalsold_totalamount_by_item.join(df_orderdetail.groupBy("Itemid").agg(sum('Totalprice').alias("TotalAmount")),on=['Itemid']).distinct()

In [307]:
totalsold_totalamount_by_item_path=f"s3a://{S3_BUCKET}/gold/TotalSold_and_TotalAmount_By_Item"
totalpay_by_user_path=f"s3a://{S3_BUCKET}/gold/TotalPay_By_User"
totalpay_by_order_path=f"s3a://{S3_BUCKET}/gold/TotalPay_By_Order"

#Load gold totalpay_by_user table
try:
      deltaTable=DeltaTable.forPath(spark,totalpay_by_user_path)
      deltaTable.alias("user").merge(
      totalpay_by_user.alias("updates"),
                      "user.Userid = updates.Userid")                                \
                      .whenMatchedUpdate(set = {
                                                'Username': "updates.Username",
                                                'Namesurname': "updates.Namesurname",
                                                'Usergender': "updates.Usergender",
                                                'Userbirthdate':"updates.Userbirthdate",
                                                'Region':"updates.Region",
                                                'City':"updates.City",
                                                'Town':"updates.Town",
                                                'District':"updates.District",
                                                'Addresstext':"updates.Addresstext",
                                                "TotalPay":"updates.TotalPay"
                                                }
                      )\
                      .whenNotMatchedInsert(values =                                                  \
                         {                      "Userid": "updates.Userid",
                                                'Username': "updates.Username",
                                                'Namesurname': "updates.Namesurname",
                                                'Usergender': "updates.Usergender",
                                                'Userbirthdate':"updates.Userbirthdate",
                                                'Region':"updates.Region",
                                                'City':"updates.City",
                                                'Town':"updates.Town",
                                                'District':"updates.District",
                                                'Addresstext':"updates.Addresstext",
                                                "TotalPay":"updates.TotalPay"}                                                                           \
                       ).execute() 
      print("Inserted and updated Totalpay by User successfully")
except:
      totalpay_by_user.write.format("delta").mode("overwrite").save(totalpay_by_user_path)
      print("Write Gold TotalPay by User table successfully")

#load gold order table
try:
      deltaTable=DeltaTable.forPath(spark,totalpay_by_order_path)
      deltaTable.alias("order").merge(
      totalpay_by_order.alias("updates"),
                      "order.Orderid = updates.Orderid")                                \
                      .whenMatchedUpdate(set = {
                                                "Userid":"updates.Userid",
                                                "Date":"updates.Date",
                                                "TotalPay":"updates.TotalPay"
                                                }
                      )\
                      .whenNotMatchedInsert(values =                                                  \
                         {                      "Orderid":"updates.Orderid",
                                                "Userid":"updates.Userid",
                                                "Date":"updates.Date",
                                                "TotalPay":"updates.TotalPay"}                                                                           \
                       ).execute() 
      print("Inserted and updated totalpay by Order successfully")
except:
      totalpay_by_order.write.format("delta").mode("overwrite").save(totalpay_by_order_path)
      print("Write gold Order table successfully")

#Load gold Item table
try:
      deltaTable=DeltaTable.forPath(spark,totalsold_totalamount_by_item_path)
      deltaTable.alias("item").merge(
      totalsold_totalamount_by_item.alias("updates"),
                      "item.Itemid = updates.Itemid")                                \
                      .whenMatchedUpdate(set = {
                                                "Itemcode": "updates.Itemcode",
                                                "Itemname":"updates.Itemname",
                                                "Amount":"updates.Amount",
                                                "Price":"updates.Price",
                                                "Totalprice":"updates.Totalprice",
                                                "Category1":"updates.Category1",
                                                "Category2":"updates.Category2",
                                                "Category3":"updates.Category3",
                                                "Category4":"updates.Category4",
                                                "Brand":"updates.Brand",
                                                "SoldQuantity":"updates.SoldQuantity",
                                                "TotalAmount":"updates.TotalAmount"
                                                }
                      )\
                      .whenNotMatchedInsert(values =                                                  \
                         {                      "Itemid": "updates.Itemid",
                                                "Itemcode": "updates.Itemcode",
                                                "Itemname":"updates.Itemname",
                                                "Amount":"updates.Amount",
                                                "Price":"updates.Price",
                                                "Totalprice":"updates.Totalprice",
                                                "Category1":"updates.Category1",
                                                "Category2":"updates.Category2",
                                                "Category3":"updates.Category3",
                                                "Category4":"updates.Category4",
                                                "Brand":"updates.brand",
                                                "SoldQuantity":"updates.SoldQuantity",
                                                "TotalAmount":"updates.TotalAmount"}                                                                           \
                       ).execute() 
      print("Inserted and updated gold Item table successfully")
except:
      totalsold_totalamount_by_item.write.format("delta").mode("overwrite").save(totalsold_totalamount_by_item_path)
      print("Write gold Item table successfully")

Write Gold TotalPay by User table successfully


Write gold Order table successfully


Write gold Item table successfully


In [308]:
df_item_gold=spark.read.format('delta').load(f"s3a://{S3_BUCKET}/gold/TotalSold_and_TotalAmount_By_Item")

In [309]:
df_totalByOrder=spark.read.format("delta").load(f"s3a://{S3_BUCKET}/gold/TotalPay_By_Order")

In [310]:
df_totalByUser=spark.read.format("delta").load(f"s3a://{S3_BUCKET}/gold/TotalPay_By_User")

In [311]:
df_bronze.select(countDistinct("ITEMID")).show()

Item_silver.select(count("Itemid")).show()

df_item_gold.select(count("Itemid")).show()

+----------------------+
|count(DISTINCT ITEMID)|
+----------------------+
|                 10837|
+----------------------+



+-------------+
|count(Itemid)|
+-------------+
|        10837|
+-------------+



+-------------+
|count(Itemid)|
+-------------+
|        10837|
+-------------+



In [312]:
df_bronze.select(countDistinct("USERID")).show()

User_silver.select(count("Userid")).show()

df_totalByUser.select(count("Userid")).show()

+----------------------+
|count(DISTINCT USERID)|
+----------------------+
|                 49801|
+----------------------+



+-------------+
|count(Userid)|
+-------------+
|        49801|
+-------------+



+-------------+
|count(Userid)|
+-------------+
|        49801|
+-------------+



In [313]:
df_bronze.select(countDistinct("ORDERID")).show()

Order_silver.select(count("Orderid")).show()

df_totalByOrder.select(count("Orderid")).show()

+-----------------------+
|count(DISTINCT ORDERID)|
+-----------------------+
|                  72077|
+-----------------------+



+--------------+
|count(Orderid)|
+--------------+
|         72077|
+--------------+



+--------------+
|count(Orderid)|
+--------------+
|         72077|
+--------------+



In [314]:
df_bronze.select(countDistinct("ORDERDETAILID")).show()

Orderdetail_silver.select(count("Orderdetailid")).show()


+-----------------------------+
|count(DISTINCT ORDERDETAILID)|
+-----------------------------+
|                       100001|
+-----------------------------+



+--------------------+
|count(Orderdetailid)|
+--------------------+
|              100001|
+--------------------+

